## Audio Analysis and Augmentation Demo <br><sup>with Interactive Plots</sup>

**Dataset:** [Cornell Birdcall Identification](https://www.kaggle.com/c/birdsong-recognition/data)


<details>
  <summary><em>Version History</em></summary>
  <br />
  <ul>
    <li>V1: <em>Initial Commit</em></li>
    <li>V2: <em>More Examples and Interactive plots</em></li>
    <li>V3: <em>Better Augmentations Demo and Comparisons</em></li>
    <li>V4: <em>Normalization and Fixes</em></li>
    <li>V5: <em>Melspec Augmentations</em></li>
  </ul>
</details>

---

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import scipy
import librosa
import librosa.display
from IPython.display import Audio
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
%matplotlib inline

In [ ]:
# Hide unnecessary warnings when loading audio files
import warnings
warnings.filterwarnings("ignore")

# Data

Load the data

In [ ]:
audio_path = '../input/birdsong-recognition/train_audio/'
test_audio_path = '../input/birdsong-recognition/example_test_audio/'

Analyse the data and the number of audio files per class

In [ ]:
subFolderList = []
for x in os.listdir(audio_path):
    if os.path.isdir(audio_path + '/' + x):
        subFolderList.append(x)

sample_audio = []
total = 0
for x in subFolderList:
    
    # get all the wave files
    all_files = [y for y in os.listdir(audio_path+x) if '.mp3' in y]
    total += len(all_files)
    
    # collect the first file from each dir
    sample_audio.append(audio_path + x + '/'+ all_files[0])
    
    # show file counts
    print(x, len(all_files), end=' | ')

In [ ]:
print('Number of classes:', len(subFolderList))
print('  Number of files:', total)

print('\nAverage number of files per class:', round(total/len(subFolderList)))

### Example Audio 

Let us consider this audio as an example to demonstrate.

In [ ]:
# demo_audio = '../input/birdsong-recognition/train_audio/aldfly/XC142068.mp3'
demo_audio = '../input/birdsong-recognition/train_audio/bewwre/XC122453.mp3' #shorter clip
demo_clip, demo_sample_rate = librosa.load(demo_audio, sr=None)
print("Class:", demo_audio.split('/')[-2])
print(" File:", demo_audio)
Audio(demo_audio)

## Audio Visualizations

### Raw waves of a few samples

Waveform is merely a graph that displays amplitude or level changes over time. Amplitude is measured in a bipolar manner, with positive and negative values.

Amazing interactive site to learn more: https://pudding.cool/2018/02/waveforms

In [ ]:
trace = [go.Scatter(
    x=np.linspace(0, demo_sample_rate/len(demo_clip), len(demo_clip)), 
    y=demo_clip
)]
layout = go.Layout(
    title = 'Waveform <br><sup>Interactive</sup>',
    yaxis = dict(title='Amplitude'),
    xaxis = dict(title='Time'),
    )
fig = go.Figure(data=trace, layout=layout)
fig.show()

This visualization is called the **time-domain** representation of a given signal. This shows us the loudness (amplitude) of sound wave changing with time. Here amplitude = 0 represents silence. These amplitudes are not very informative, as they only talk about the loudness of audio recording. 

In [ ]:
# fig = plt.figure(figsize=(25,15))
fig = plt.figure(figsize=(25,12))
for i, filepath in enumerate(sample_audio[:20]):
    # plt.subplot(5,1,i+1)
    plt.subplot(5,4,i+1)
    clip, sample_rate = librosa.load(filepath, sr=None)
    plt.title(filepath.split('/')[-2])
    plt.axis('off')
    plt.plot(clip, c='black', lw=0.5)

In [ ]:
def plot_raw_waves(label, color=None):
    same_samples = [audio_path + f'{label}/' + y for y in os.listdir(audio_path + f'{label}/')[:20]]

    fig = plt.figure(figsize=(25,12))
    fig.suptitle(label, fontsize=30, c=color)
    for i, filepath in enumerate(same_samples):
        plt.subplot(5,4,i+1)
        clip, sample_rate = librosa.load(filepath, sr=None)
        plt.axis('off')
        plt.plot(clip, c=color, lw=0.5)

In [ ]:
plot_raw_waves('ribgul', '#FBC02D')

In [ ]:
plot_raw_waves('casfin', '#D81B60')

In [ ]:
plot_raw_waves('snobun', '#F4511E')

In [ ]:
plot_raw_waves('lazbun', '#039BE5')

### Spectrograms of a few samples

A spectrogram is a visual way of representing the signal strength, or “loudness”, of a signal over time at various frequencies present in a particular waveform.  Not only can one see whether there is more or less energy at, for example, 2 Hz vs 10 Hz, but one can also see how energy levels vary over time.

Spectrograms can be two-dimensional graphs with a third variable represented by color. One axis represents time, and the other axis represents frequency; a third dimension indicating the amplitude of a particular frequency at a particular time is represented by the intensity or color of each point in the image.

Amazing spectrogram visualization for ***bird*** here: https://musiclab.chromeexperiments.com/Spectrogram/

Function that calculates spectrogram.

Note, that we are taking logarithm of spectrogram values. It will make our plot much more clear, moreover, it is strictly connected to the way people hear. We need to assure that there are no 0 values as input to logarithm.

In [ ]:
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = scipy.signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

In [ ]:
demo_freqs, demo_times, demo_spec = log_specgram(demo_clip, sample_rate)

In [ ]:
trace = [go.Heatmap(
    x= demo_times,
    y= demo_freqs,
    z= demo_spec.T,
    colorscale='viridis',
    )]
layout = go.Layout(
    title = 'Spectrogram <br><sup>Interactive</sup>',
    yaxis = dict(title='Frequency'),
    xaxis = dict(title='Time'),
    )
fig = go.Figure(data=trace, layout=layout)
fig.show()

In [ ]:
fig = plt.figure(figsize=(25,12))
for i, filepath in enumerate(sample_audio[:20]):
    plt.subplot(5,4,i+1)
    label = filepath.split('/')[-2]
    plt.title(label)
    clip, sample_rate = librosa.load(filepath, sr=None)
    _, _, spectrogram = log_specgram(clip, sample_rate)
    plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    plt.axis('off');

In [ ]:
def plot_spectrograms(label):
    same_samples = [audio_path + f'{label}/' + y for y in os.listdir(audio_path + f'{label}/')[:20]]

    fig = plt.figure(figsize=(25,12))
    fig.suptitle(label, fontsize=30)
    for i, filepath in enumerate(same_samples):
        plt.subplot(5,4,i+1)
        clip, sample_rate = librosa.load(filepath, sr=None)
        _, _, spectrogram = log_specgram(clip, sample_rate)
        plt.imshow(spectrogram.T, aspect='auto', origin='lower')
        plt.axis('off')

In [ ]:
plot_spectrograms('ribgul')

In [ ]:
plot_spectrograms('casfin')

In [ ]:
plot_spectrograms('snobun')

In [ ]:
plot_spectrograms('lazbun')

### 3D Spectrogram

In [ ]:
trace = [go.Surface(
    x= demo_times,
    y= demo_freqs,
    z= demo_spec.T,
    colorscale='viridis',
)]
layout = go.Layout(
title='3D Specgtrogram <br><sup>Interactive</sup>',
scene = dict(
    yaxis = dict(title='Frequency', range=[demo_freqs.min(),demo_freqs .max()]),
    xaxis = dict(title='Time', range=[demo_times.min(),demo_times.max()],),
    zaxis = dict(title='Log amplitude'),
    ),
)
fig = go.Figure(data=trace, layout=layout)
fig.show()

## Fast Fourier Transform (FFT)

An audio signal is a complex signal composed of multiple ‘single-frequency sound waves’ which travel together as a disturbance(pressure-change) in the medium. When sound is recorded we only capture the resultant amplitudes of those multiple waves. Fourier Transform is a mathematical concept that can **decompose a signal into its constituent frequencies**. Fourier transform does not just give the frequencies present in the signal, It also gives the magnitude of each frequency present in the signal. The only difference between FT(Fourier Transform) and FFT is that FT considers a continuous signal while FFT takes a discrete signal as input.

Function that calculates FFT.

Note, FFT is simmetrical, so we take just the first half and FFT is also complex, so we consider the real part (abs).

In [ ]:
from scipy.fftpack import fft
def custom_fft(y, fs):
    T = 1.0 / fs
    N = y.shape[0]
    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    vals = 2.0/N * np.abs(yf[0:N//2])  
    return xf, vals

In [ ]:
demo_xf, demo_vals = custom_fft(demo_clip, demo_sample_rate)

In [ ]:
trace = [go.Scatter(
    x=demo_xf, 
    y=demo_vals,
    line_color='deeppink'
)]
layout = go.Layout(
    title = 'Fast Fourier Transform (FFT) <br><sup>Interactive</sup>',
    yaxis = dict(title='Magnitude'),
    xaxis = dict(title='Frequency'),
    )
fig = go.Figure(data=trace, layout=layout)
fig.show()

In [ ]:
fig = plt.figure(figsize=(25,12))
for i, filepath in enumerate(sample_audio[:20]):
    plt.subplot(5,4,i+1)
    label = filepath.split('/')[-2]
    plt.title(label)
    clip, sample_rate = librosa.load(filepath, sr=None)
    xf, vals = custom_fft(clip, sample_rate)
    plt.plot(xf, vals, c='black')
    plt.axis('off');

In [ ]:
def plot_ffts(label, color=None):
    same_samples = [audio_path + f'{label}/' + y for y in os.listdir(audio_path + f'{label}/')[:20]]

    fig = plt.figure(figsize=(25,12))
    fig.suptitle(label, fontsize=30, c=color)
    for i, filepath in enumerate(same_samples):
        plt.subplot(5,4,i+1)
        clip, sample_rate = librosa.load(filepath, sr=None)
        xf, vals = custom_fft(clip, sample_rate)
        plt.plot(xf, vals, c=color)
        plt.axis('off')

In [ ]:
plot_ffts('ribgul', '#FBC02D')

In [ ]:
plot_ffts('casfin', '#D81B60')

In [ ]:
plot_ffts('snobun', '#F4511E')

In [ ]:
plot_ffts('lazbun', '#039BE5')

## Mel Spectrogram

The Mel Scale, mathematically speaking, is the result of some **non-linear transformation of the frequency scale**. This Mel Scale is constructed such that sounds of equal distance from each other on the Mel Scale, also _“sound”_ to humans as they are equal in distance from one another.
In contrast to Hz scale, where the difference between 500 and 1000 Hz is obvious, whereas the difference between 7500 and 8000 Hz is barely noticeable.
Mel Spectrogram, is, rather surprisingly, a Spectrogram with the Mel Scale as its y axis.
More [here](https://towardsdatascience.com/getting-to-know-the-mel-spectrogram-31bca3e2d9d0).


In [ ]:
demo_S = librosa.feature.melspectrogram(demo_clip, sr=demo_sample_rate, n_mels=128)    
demo_log_S = librosa.power_to_db(demo_S, ref=np.max)

In [ ]:
trace = [go.Heatmap(
    x= demo_times,
    y= demo_freqs,
    z= demo_log_S,
    colorscale='magma',
    )]
layout = go.Layout(
    title = 'Mel Power Spectrogram <br><sup>Interactive</sup>',
    yaxis = dict(title='Mel'),
    xaxis = dict(title='Time'),
    )
fig = go.Figure(data=trace, layout=layout)
fig.show()

In [ ]:
fig = plt.figure(figsize=(25,12))
for i, filepath in enumerate(sample_audio[:20]):
    plt.subplot(5,4,i+1)
    label = filepath.split('/')[-2]
    plt.title(label)
    clip, sample_rate = librosa.load(filepath, sr=None)
    S = librosa.feature.melspectrogram(clip, sr=sample_rate, n_mels=128)    
    log_S = librosa.power_to_db(S, ref=np.max)    
    librosa.display.specshow(log_S, sr=sample_rate, x_axis='time', y_axis='mel')
    plt.axis('off');

In [ ]:
def plot_mel_spectrograms(label):
    same_samples = [audio_path + f'{label}/' + y for y in os.listdir(audio_path + f'{label}/')[:20]]

    fig = plt.figure(figsize=(25,12))
    fig.suptitle(label, fontsize=30)
    for i, filepath in enumerate(same_samples):
        plt.subplot(5,4,i+1)
        clip, sample_rate = librosa.load(filepath, sr=None)
        S = librosa.feature.melspectrogram(clip, sr=sample_rate, n_mels=128)    
        log_S = librosa.power_to_db(S, ref=np.max)    
        librosa.display.specshow(log_S, sr=sample_rate, x_axis='time', y_axis='mel')
        plt.axis('off');

In [ ]:
plot_mel_spectrograms('ribgul')

In [ ]:
plot_mel_spectrograms('casfin')

In [ ]:
plot_mel_spectrograms('snobun')

In [ ]:
plot_mel_spectrograms('lazbun')

## Mel Frequency Cepstral Coefficient (MFCC)

MFCC features represent phonemes (distinct units of sound) as the shape of the vocal tract (which is responsible for sound generation) is manifest in them. The mel frequency cepstral coefficients (MFCCs) of a signal are a small set of features (usually about 10-20) which concisely **describe the overall shape of a spectral envelope**.
More [here](https://medium.com/prathena/the-dummys-guide-to-mfcc-aceab2450fd) & [here](http://practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/).

> In classical, but still state-of-the-art systems, MFCC or similar features are taken as the input to the system instead of spectrograms. However, in end-to-end (often neural-network based) systems, the most common input features are probably raw spectrograms, or mel power spectrograms. For example MFCC decorrelates features, but NNs deal with correlated features well.

In [ ]:
demo_mfcc = librosa.feature.mfcc(S=demo_log_S, n_mfcc=13)
demo_delta2_mfcc = librosa.feature.delta(demo_mfcc, order=2)

In [ ]:
trace = [go.Heatmap(
    x= demo_times,
    y= demo_freqs,
    z= demo_delta2_mfcc,
    colorscale='RdBu_r',
    )]
layout = go.Layout(
    title = 'MFCC  <br><sup>Interactive</sup>',
    yaxis = dict(title='MFCC coeffs'),
    xaxis = dict(title='Time'),
    )
fig = go.Figure(data=trace, layout=layout)
fig.show()

In [ ]:
fig = plt.figure(figsize=(25,12))
for i, filepath in enumerate(sample_audio[:20]):
    plt.subplot(5,4,i+1)
    label = filepath.split('/')[-2]
    plt.title(label)
    clip, sample_rate = librosa.load(filepath, sr=None)
    S = librosa.feature.melspectrogram(clip, sr=sample_rate, n_mels=128)    
    log_S = librosa.power_to_db(S, ref=np.max)    
    mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    librosa.display.specshow(delta2_mfcc, cmap='bwr')
    plt.axis('off');

In [ ]:
def plot_mmfcs(label):
    same_samples = [audio_path + f'{label}/' + y for y in os.listdir(audio_path + f'{label}/')[:20]]

    fig = plt.figure(figsize=(25,12))
    fig.suptitle(label, fontsize=30)
    for i, filepath in enumerate(same_samples):
        plt.subplot(5,4,i+1)
        clip, sample_rate = librosa.load(filepath, sr=None)
        S = librosa.feature.melspectrogram(clip, sr=sample_rate, n_mels=128)    
        log_S = librosa.power_to_db(S, ref=np.max)    
        mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
        delta2_mfcc = librosa.feature.delta(mfcc, order=2)
        librosa.display.specshow(delta2_mfcc, cmap='bwr')
        plt.axis('off');

In [ ]:
plot_mmfcs('ribgul')

In [ ]:
plot_mmfcs('casfin')

In [ ]:
plot_mmfcs('snobun')

In [ ]:
plot_mmfcs('lazbun')

## In-depth per sample analysis

Let's analyse individual audio files with all the above visualizations to get a better understanding.

In [ ]:
def audio_plots(filename):
    clip, sample_rate = librosa.load(filename, sr=None)
    freqs, times, spectrogram = log_specgram(clip, sample_rate)
    label = filename.split('/')[-2]
    
    # # Normalize (if needed)
    # mean = np.mean(spectrogram, axis=0)
    # std = np.std(spectrogram, axis=0)
    # spectrogram = (spectrogram - mean) / std

    fig = plt.figure(figsize=(20,15), dpi=200)
      
    # Raw wave
    ax1 = fig.add_subplot(511)
    ax1.set_title('Raw wave of ' + label)
    ax1.set_ylabel('Amplitude')
    librosa.display.waveplot(clip.astype('float'), sr=sample_rate)

    # FFT    
    ax2 = fig.add_subplot(512)
    xf, vals = custom_fft(clip, sample_rate)
    ax2.set_title('FFT of ' + label + ' with ' + str(sample_rate) + ' Hz')
    ax2.plot(xf, vals, 'm')
    ax2.set_xlabel('Frequency')
    ax2.set_ylabel('Magnitude')
    ax2.grid()
    
    # Spectrogram    
    ax3 = fig.add_subplot(513)
    ax3.set_title('Spectrogram of ' + label)
    ax3.set_ylabel('Freqs in Hz')
    ax3.set_xlabel('Seconds')
    ax3.imshow(spectrogram.T, aspect='auto', origin='lower', 
               extent=[times.min(), times.max(), freqs.min(), freqs.max()])
    
    # Mel power spectrogram
    ax4 = fig.add_subplot(514)    
    S = librosa.feature.melspectrogram(clip, sr=sample_rate, n_mels=128)    
    log_S = librosa.power_to_db(S, ref=np.max) 
    librosa.display.specshow(log_S, sr=sample_rate, x_axis='time', y_axis='mel')
    ax4.set_title('Mel power spectrogram of ' + label)   
    
    # MFCC
    ax5 = fig.add_subplot(515)
    mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    librosa.display.specshow(delta2_mfcc, cmap='bwr')
    ax5.set_title('MFCC of ' + label)
    ax5.set_ylabel('MFCC coeffs')
    ax5.set_xlabel('Time')

    plt.tight_layout()
    print(f"Class: {label}\n File: {filename}, ")
    return Audio(filename)

In [ ]:
audio_plots(demo_audio)

In [ ]:
audio_plots(sample_audio[1])

In [ ]:
audio_plots(sample_audio[50])

In [ ]:
audio_plots(sample_audio[100])

In [ ]:
audio_plots(sample_audio[150])

In [ ]:
audio_plots(sample_audio[200])

In [ ]:
audio_plots(sample_audio[250])

## Audio Augmentations

Let us consider this *(same)* audio as an example to demonstrate.

> ***NOTE:*** Augmentation values are boosted to showcase their effect

In [ ]:
def load_audio_file(filename):
    data, sr = librosa.load(filename, sr=None)
    return data, sr

sns.set(style='darkgrid')
def plot_time_series(data, sr, alpha=0.5, color=None, label=None):
#     plt.figure(figsize=(20,5))
    plt.title('Raw wave')
    plt.ylabel('Amplitude')
    plt.plot(np.linspace(0, sr/len(data), len(data)), data, alpha=alpha, c=color, label=label)
    plt.legend(loc='lower left', ncol=2, frameon=False)
    return Audio(data, rate=sr)

def plot_spectrogram(data, sr, label=None):
    S = librosa.feature.melspectrogram(data, sr, n_mels=128)    
    log_S = librosa.power_to_db(S, ref=np.max)
    plt.figure(figsize=(20,3), dpi=200)
    plt.title('Mel power spectrogram')
    plt.text(0.05, 200, label, fontsize=12, c='w')
    librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')    
#     plt.axis('off')

In [ ]:
org_data, sr = load_audio_file(demo_audio)
print("Class:", demo_audio.split('/')[-2])
print(" File:", demo_audio)

plt.figure(figsize=(20,5))
plot_time_series(org_data, sr, 0.67, 'blue', 'Actual')

<span style="color: blue;"><strong>BLUE</strong></span> is the actual audio clip and <span style="color: red;"><strong>RED</strong></span> is for the augmented audio clip.

In [ ]:
plot_spectrogram(org_data, sr, 'Actual')

### Normalize

Let's first normalize the audio clip before applying augmentations.

In [ ]:
# Use mean & std of individual data or of all data as custom params
def normalize(X, mean=None, std=None):
    mean = mean or X.mean()
    std = std or (X-X.mean()).std()
    return ((X - mean)/std).astype(np.float64)

In [ ]:
data = normalize(org_data)

In [ ]:
plt.figure(figsize=(20,5))
plot_time_series(org_data, sr, 0.7, 'blue', 'Actual')
plot_time_series(data, sr, 0.6, 'red', 'Normalized')

In [ ]:
plot_spectrogram(org_data, sr, 'Actual')
plot_spectrogram(data, sr, 'Normalized')

### Adding white noise 

In [ ]:
wn = np.random.randn(len(data))
data_wn = data + 0.1 * wn

In [ ]:
plt.figure(figsize=(20,5))
plot_time_series(data, sr, 0.7, 'blue', 'Actual')
plot_time_series(data_wn, sr, 0.6, 'red', 'White Noise')

In [ ]:
plot_spectrogram(data, sr, 'Actual')
plot_spectrogram(data_wn, sr, 'White Noise')

### Shifting the sound

In [ ]:
data_roll = np.roll(data, 5000)

In [ ]:
plt.figure(figsize=(20,5))
plot_time_series(data, sr, 0.5, 'blue', 'Actual')
plot_time_series(data_roll, sr, 0.7, 'red', 'Shifted')

In [ ]:
plot_spectrogram(data, sr, 'Actual')
plot_spectrogram(data_roll, sr, 'Shifted')

### Stretching the sound

In [ ]:
# larger value
data_stretch = librosa.effects.time_stretch(data, 1.2)

In [ ]:
plt.figure(figsize=(20,5))
plot_time_series(data, sr, 0.3, 'blue', 'Actual')
plot_time_series(data_stretch, sr, 0.8, 'red', 'Stretched')

In [ ]:
plot_spectrogram(data, sr, 'Actual')
plot_spectrogram(data_stretch, sr, 'Stretched')

In [ ]:
# smaller value
data_stretch = librosa.effects.time_stretch(data, 0.8)

In [ ]:
plt.figure(figsize=(20,5))
plot_time_series(data, sr, 0.3, 'blue', 'Actual')
plot_time_series(data_stretch, sr, 0.8, 'red', 'Stretched')

In [ ]:
plot_spectrogram(data, sr, 'Actual')
plot_spectrogram(data_stretch, sr, 'Stretched')

### Changing the Pitch

In [ ]:
data_pitch = librosa.effects.pitch_shift(data, sr, n_steps=-10)

In [ ]:
plt.figure(figsize=(20,5))
plot_time_series(data, sr, 0.5, 'blue', 'Actual')
plot_time_series(data_pitch, sr, 0.8, 'red', 'Pitch changed')

In [ ]:
plot_spectrogram(data, sr, 'Actual')
plot_spectrogram(data_pitch, sr, 'Pitch changed')

### Inverting the Polarity

In [ ]:
data_invert = -data

In [ ]:
plt.figure(figsize=(20,5))
plot_time_series(data, sr, 1, 'blue', 'Actual')
plot_time_series(data_invert, sr, 0.7, 'red', 'Inverted')

In [ ]:
plot_spectrogram(data, sr, 'Actual')
plot_spectrogram(data_invert, sr, 'Inverted')

### All Augmentations

In [ ]:
x = np.linspace(0, sr/len(data), len(data))
trace = [
    go.Scatter(x=x, y=org_data, name='Actual', opacity=0.5),
    go.Scatter(x=x, y=data, name='Normalized', opacity=0.5),
    go.Scatter(x=x, y=data_wn, name='Noise', opacity=0.5),
    go.Scatter(x=x, y=data_roll, name='Shift', opacity=0.5),
    go.Scatter(x=x, y=data_stretch, name='Stretch', opacity=0.5),
    go.Scatter(x=x, y=data_pitch, name='Pitch', opacity=0.5),
    go.Scatter(x=x, y=data_invert, name='Invert', opacity=0.5),
]

layout = go.Layout(
    title = 'Augmentations <br><sup>Interactive</sup>',
    yaxis = dict(title='Amplitude'),
    xaxis = dict(title='Time'),
    legend_title_text='Augmentations <br><sup>Toggle augmentations on/off</sup>'
    )

fig = go.Figure(data=trace, layout=layout)
fig.show()

#### Combining all in one

In [ ]:
print("ACTUAL:")
org_data, sr = load_audio_file(demo_audio)
Audio(org_data, rate=sr)

In [ ]:
data = normalize(org_data)
data_wn = data + 0.1 * wn
data_roll = np.roll(data_wn, 5000)
data_stretch = librosa.effects.time_stretch(data_roll, 1)
data_pitch = librosa.effects.pitch_shift(data_stretch, sr, n_steps=-10)
data_invert = -data_pitch
data_aug = data_invert

plt.figure(figsize=(20,5))
plot_time_series(data, sr, 0.3, 'blue', 'Actual')
print("AUGMENTED:")
plot_time_series(data_aug, sr, 0.7, 'red', 'Augmented')

In [ ]:
plot_spectrogram(data, sr, 'Actual')
plot_spectrogram(data_aug, sr, 'Augmented')

## Mel Spec Augmentations

As mentioned earlier, we can convert this time-series problem into a computer vision problem by converting the audio data into mel spectrograms *(or MFCC)*.

This is similar to any other image classification problem, where all the common augmentations can be done. Here are a few most common augmentation methods for mel spectrograms.

Let us consider this *(same)* audio as an example to demonstrate.

In [ ]:
clip, sample_rate = librosa.load(demo_audio, sr=None)
print("Class:", demo_audio.split('/')[-2])
print(" File:", demo_audio)
Audio(demo_audio)

In [ ]:
S = librosa.feature.melspectrogram(clip, sr=sample_rate, n_mels=256)    
log_S = librosa.power_to_db(S, ref=np.max)

plt.figure(figsize=(20,4))
plt.title('Mel power spectrogram')
plt.text(0.05, 200, 'original audio', fontsize=12, c='w')
librosa.display.specshow(log_S, sr=sample_rate, x_axis='time', y_axis='mel');

### Spec Dropout

We augment spectrograms by hiding random time & frequency intervals.

In [ ]:
def spec_augment(spec: np.ndarray, num_mask=2, freq_masking_max_percentage=0.15, time_masking_max_percentage=0.3):
    spec = spec.copy()
    for i in range(num_mask):
        num_freqs, num_frames = spec.shape
        freq_percentage = random.uniform(0.0, freq_masking_max_percentage)
        time_percentage = random.uniform(0.0, time_masking_max_percentage)
        
        num_freqs_to_mask = int(freq_percentage * num_freqs)
        num_frames_to_mask = int(time_percentage * num_frames)
        
        t0 = int(np.random.uniform(low=0.0, high=num_frames - num_frames_to_mask))
        f0 = int(np.random.uniform(low=0.0, high=num_freqs - num_freqs_to_mask))
        
        spec[:, t0:t0 + num_frames_to_mask] = 0     
        spec[f0:f0 + num_freqs_to_mask, :] = 0 
        
    return spec

In [ ]:
for i in range(4):
    plt.figure(figsize=(20,3))
    librosa.display.specshow(spec_augment(log_S), sr=sample_rate, x_axis='time', y_axis='mel');
#     plt.axis('off')

### Pad to size

If the spectrogram is shorter than the required image size, it can be padded.

In [ ]:
def pad_to_size(signal, size, mode):
    if signal.shape[1] < size:
        padding = size - signal.shape[1]
        offset = padding // 2
        pad_width = ((0, 0), (offset, padding - offset))
        if mode == 'constant':
            signal = np.pad(signal, pad_width, 'constant', constant_values=signal.min())
        elif mode == 'wrap':
            signal = np.pad(signal, pad_width, 'wrap')
    return signal    

In [ ]:
img_size = 400

plt.figure(figsize=(20,4))
plt.title('Mel power spectrogram')
plt.text(0.05, 200, 'constant', fontsize=12, c='w')
librosa.display.specshow(pad_to_size(log_S, img_size, 'constant'), sr=sample_rate, x_axis='time', y_axis='mel');

plt.figure(figsize=(20,4))
plt.title('Mel power spectrogram')
plt.text(0.05, 200, 'wrap', fontsize=12, c='w')
librosa.display.specshow(pad_to_size(log_S, img_size, 'wrap'), sr=sample_rate, x_axis='time', y_axis='mel');

## Acknowledgements and References:

- https://www.kaggle.com/davids1992/speech-representation-and-data-exploration
- https://www.kaggle.com/timolee/audio-data-conversion-to-images-eda
- https://www.kaggle.com/CVxTz/audio-data-augmentation

---